In [1]:
from os import listdir
from os.path import join, isfile
import numpy as np #1.16.4 otherwise futurewarning tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
from sklearn.model_selection import LeaveOneOut
import cv2
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import Normalizer

In [2]:
NUM_CLASSES = 2
BATCH_SIZE = 8
NUM_POINTS = 270

# Data

In [3]:
def load_data(syn):
    
    # files with representations
    GENERAL_DIR = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien" 
    syn_csv = GENERAL_DIR+ "\\features_facereader_landmarks_patient_groups.csv"
    ID_csv = GENERAL_DIR+ "\\features_facereader_landmarks_all_controls.csv"
    
    # open directories
    syn_dir = GENERAL_DIR+"\\{}\\{}-patients".format(syn, syn)
    ID_dir = GENERAL_DIR+ "\\{}\\{}-selected-ID-controls".format(syn, syn)

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f))) and ".jpg" in f]
    files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f))) and ".jpg" in f]
    
    data, labels, data_syn, data_ID = [], [], [], []

    with open (syn_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_syn:                 
                fr_rep = [float(f) for f in row[1:]]
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append([float(row[i]), float(row[i+1]), float(row[i+2])])
                    i+=3                       
                data_syn.append(rep)
                
    with open (ID_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_ID:                 
                fr_rep = [float(f) for f in row[1:]]
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append([float(row[i]), float(row[i+1]), float(row[i+2])])
                    i+=3                       
                data_ID.append(rep)

    for index, (syn_item, ID_item) in enumerate(zip(data_syn, data_ID)):
        if len(syn_item) == 510 and len(ID_item) == 510:               
            data.append(syn_item)
            labels.append(1)
            data.append(ID_item)
            labels.append(0)

    return np.array(data), np.array(labels)

In [4]:
def augment(points, label):
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    points = tf.random.shuffle(points)
    return points, label

In [5]:
def split_data(data, left, right):
    data_left, data_right  = [], []
    for index, row in enumerate(data):
        
        rep_left, rep_right = [], []
        for p, [x, y, z] in enumerate(row):
            
            if p in left:
                rep_left.append([x, y, z])

            if p in right:
                rep_right.append([x, y, z])
                
        data_left.append(rep_left)
        data_right.append(rep_right)
        
    return np.array(data_left), np.array(data_right)

# Model

In [6]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding='valid')(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

In [7]:
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)
        
    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2,2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

In [8]:
def tnet(inputs, num_features):
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)
    
    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features, 
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg)(x)
    
    feat_T = layers.Reshape((num_features, num_features))(x)
    
    return layers.Dot(axes=(2,1))([inputs, feat_T])

In [9]:
def generate_model():

    inputs = keras.Input(shape=(NUM_POINTS, 3))
    x = tnet(inputs, 3)
    x = conv_bn(x, 32)
    x = conv_bn(x, 32)
    x = tnet(x, 32)
    x = conv_bn(x, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = layers.Dropout(rate=0.3)(x)
    x = dense_bn(x, 128)
    x = layers.Dropout(rate=0.3)(x)

    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Adam(lr=0.001),
        metrics=["sparse_categorical_accuracy"])
    
    return model

In [10]:
left = [0, 1, 2, 3, 4, 5, 6, 7, 8, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341]
right = [9, 10, 11, 12, 13, 14, 15, 16, 17, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 157, 158, 168, 173, 174, 190, 191, 205, 210, 213, 217, 262, 263, 269, 278, 281, 282, 284, 288, 292, 294, 295, 296, 297, 298, 299, 302, 304, 305, 306, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509]

# Training

In [11]:
run_nr = 1
side = ["LEFT", "RIGHT"]
syn_list = ['ADNP', 'ANKRD11', 'CDK13', 'DEAF1', 'DYRK1A', 'EHMT1', 'FBXO11', 'SON', 'WAC', 'YY1', 'KDVS']

results_file = open("results/pointnet_results_binary_run_split_faces{}.txt".format(run_nr), "w")

for syn in syn_list:
    data, labels = load_data(syn)
    data_left, data_right = split_data(data, left, right)

    for s, data in enumerate([data_left, data_right]):
        results_file.write("Syndrome {} with {} patients and {} controls - {}\n".format(syn, labels.tolist().count(1), labels.tolist().count(0), side[s]))
        all_y, all_probs, all_preds = [], [], [] 

        loo = LeaveOneOut()
        for train_index, test_index in tqdm(loo.split(data)):
            X_train, X_test = np.array(data[train_index]), data[test_index]
            y_train, y_test = np.array(labels[train_index]), labels[test_index]

            model = generate_model()
            model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=10, shuffle=True)

            y_pred_array = model.predict(X_test)
            y_pred = tf.math.argmax(y_pred_array, -1).numpy()

            all_y.append(y_test[0])
            all_probs.append(y_pred_array[0][1])
            all_preds.append(y_pred) 

        aroc = roc_auc_score(all_y, all_probs)
        tn, fp, fn, tp = confusion_matrix(all_y, all_preds).ravel()
        spec = tn / (tn+fp)  
        sens = tp / (tp+fn)

        results_file.write("AROC: {:.4f}, spec: {:.4f}, sens: {:.4f}\n\n".format(aroc, spec, sens))
    
results_file.close()

0it [00:00, ?it/s]

Epoch 1/10
3/3 [==============================] - 0s 77ms/step - loss: 1.6049 - sparse_categorical_accuracy: 0.3043
Epoch 2/10
3/3 [==============================] - 0s 87ms/step - loss: 1.1502 - sparse_categorical_accuracy: 0.8261
Epoch 3/10
3/3 [==============================] - 0s 89ms/step - loss: 1.1826 - sparse_categorical_accuracy: 0.7826
Epoch 4/10
3/3 [==============================] - 0s 84ms/step - loss: 1.2601 - sparse_categorical_accuracy: 0.6957
Epoch 5/10
3/3 [==============================] - 0s 78ms/step - loss: 1.0624 - sparse_categorical_accuracy: 0.8696
Epoch 6/10
3/3 [==============================] - 0s 78ms/step - loss: 0.9984 - sparse_categorical_accuracy: 0.9130
Epoch 7/10
3/3 [==============================] - 0s 78ms/step - loss: 1.0832 - sparse_categorical_accuracy: 0.9130
Epoch 8/10
3/3 [==============================] - 0s 83ms/step - loss: 0.8673 - sparse_categorical_accuracy: 0.9565
Epoch 9/10
3/3 [==============================] - 0s 79ms/step - loss: 0

1it [00:09,  9.50s/it]

Epoch 1/10
3/3 [==============================] - 0s 80ms/step - loss: 1.6729 - sparse_categorical_accuracy: 0.4348
Epoch 2/10
3/3 [==============================] - 0s 78ms/step - loss: 1.0013 - sparse_categorical_accuracy: 0.8696
Epoch 3/10
3/3 [==============================] - 0s 79ms/step - loss: 0.8979 - sparse_categorical_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 0s 73ms/step - loss: 0.9328 - sparse_categorical_accuracy: 0.9565
Epoch 5/10
3/3 [==============================] - 0s 90ms/step - loss: 0.8847 - sparse_categorical_accuracy: 0.9565
Epoch 6/10
3/3 [==============================] - 0s 69ms/step - loss: 0.7264 - sparse_categorical_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 78ms/step - loss: 0.7962 - sparse_categorical_accuracy: 0.8696
Epoch 8/10
3/3 [==============================] - 0s 78ms/step - loss: 0.5757 - sparse_categorical_accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 0s 78ms/step - loss: 0

2it [00:18,  9.39s/it]

...

102it [1:38:27, 54.75s/it]

Epoch 1/10
13/13 [==============================] - 4s 284ms/step - loss: 1.8758 - sparse_categorical_accuracy: 0.4951
Epoch 2/10
13/13 [==============================] - 4s 283ms/step - loss: 1.6491 - sparse_categorical_accuracy: 0.5437
Epoch 3/10
13/13 [==============================] - 4s 282ms/step - loss: 1.4995 - sparse_categorical_accuracy: 0.5049
Epoch 4/10
13/13 [==============================] - 4s 287ms/step - loss: 1.3243 - sparse_categorical_accuracy: 0.4951
Epoch 5/10
13/13 [==============================] - 4s 281ms/step - loss: 1.1744 - sparse_categorical_accuracy: 0.5631
Epoch 6/10
13/13 [==============================] - 4s 284ms/step - loss: 1.1708 - sparse_categorical_accuracy: 0.5728
Epoch 7/10
13/13 [==============================] - 4s 283ms/step - loss: 1.1068 - sparse_categorical_accuracy: 0.5728
Epoch 8/10
13/13 [==============================] - 4s 283ms/step - loss: 1.1444 - sparse_categorical_accuracy: 0.5534
Epoch 9/10
13/13 [==============================

103it [1:39:19, 53.85s/it]

Epoch 1/10
13/13 [==============================] - 4s 283ms/step - loss: 1.7724 - sparse_categorical_accuracy: 0.4660
Epoch 2/10
13/13 [==============================] - 4s 279ms/step - loss: 1.6333 - sparse_categorical_accuracy: 0.5437
Epoch 3/10
13/13 [==============================] - 4s 280ms/step - loss: 1.6301 - sparse_categorical_accuracy: 0.4951
Epoch 4/10
13/13 [==============================] - 4s 279ms/step - loss: 1.3378 - sparse_categorical_accuracy: 0.6117
Epoch 5/10
13/13 [==============================] - 4s 279ms/step - loss: 1.3424 - sparse_categorical_accuracy: 0.5049
Epoch 6/10
13/13 [==============================] - 4s 281ms/step - loss: 1.4436 - sparse_categorical_accuracy: 0.4854
Epoch 7/10
13/13 [==============================] - 4s 281ms/step - loss: 1.2787 - sparse_categorical_accuracy: 0.5243
Epoch 8/10
13/13 [==============================] - 4s 280ms/step - loss: 1.2849 - sparse_categorical_accuracy: 0.5534
Epoch 9/10
13/13 [==============================

104it [1:40:10, 57.79s/it]
